In [1]:
import mysql.connector as connector
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
from mysql.connector import errors

# Task1

In [2]:
dbconfig = {
    'database':'little_lemon_db',
    'user':'root',
    'password':''
    
}
try:
    pool = MySQLConnectionPool(pool_name="pool_b",
                          pool_size=2,
                          host="127.0.0.1",**dbconfig
                          )
    print("The connection pool is created with a name {}".format(pool.pool_name))
    print("The pool size is: {}".format(pool.pool_size))
        
except Error as err:
    print("Error code: {}".format(err.errno))
    print("Error message: {}".format(err.msg))

The connection pool is created with a name pool_b
The pool size is: 2


# Task 2

In [3]:
data = [[8,'Anees','Java','18:00:00',6],[5,'Bald','Vin','19:00:00',6],[12,'Jay','Kon','19:30:00',6]]
for item in data:
    try:
        query = """INSERT INTO bookings(TableNo,GuestFirstName,GuestLastName,BookingSlot,EmployeeID) VALUES ({},'{}','{}','{}',{});""".format(item[0],item[1],item[2],item[3],item[4])
        connection_object = pool.get_connection()
        # use the connection
        if connection_object.is_connected:
            cursor = connection_object.cursor()
            cursor.execute(query)
            connection_object.commit()
    except errors.PoolError as err:
        print("Error: {}".format(err))
        print("Adding new connection to the pool")
        cnx = connector.connect(user="root",password="")
        pool.add_connection(cnx=cnx)
        conn = pool.get_connection()
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()
connection_object.close()
conn.close()
        

    

Error: Failed getting connection; pool exhausted
Adding new connection to the pool


# Task 3.1

In [4]:
connection = pool.get_connection()
#connection = connector.connect(user="root",password="")
cursor = connection.cursor()
cursor.execute("USE little_lemon_db;")
query1 = """SELECT EmployeeID,Name FROM Employees WHERE Role='Manager';"""
cursor.execute(query1)
results = cursor.fetchall()
columns = cursor.column_names
print(columns)
for item in results:
    print(item)

('EmployeeID', 'Name')
(1, 'Mario Gollini')


# Task 3.2

In [5]:
query2 = "SELECT Name,Role FROM Employees ORDER BY Annual_Salary DESC LIMIT 1;"
cursor.execute(query2)
columns = cursor.column_names
results = cursor.fetchall()
print(columns)
for item in results:
    print(item)

('Name', 'Role')
('Mario Gollini', 'Manager')


# Task 3.3

In [6]:
query3 = """SELECT COUNT(*) NumOFBooking FROM bookings WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';"""
cursor.execute(query3)
columns = cursor.column_names
results = cursor.fetchall()
print(columns)
for item in results:
    print(item)

('NumOFBooking',)
(7,)


# Task 3.4

In [7]:
query4 = """SELECT B.BookingID,CONCAT(B.GuestFirstName,' ',B.GuestLastName) AS FullName FROM
            Bookings B JOIN employees E ON B.EmployeeID = E.EmployeeID
            WHERE E.Role='Receptionist' ORDER BY B.BookingSlot;"""
cursor.execute(query4)
columns = cursor.column_names
results = cursor.fetchall()
print(columns)
for item in results:
    print(item)

('BookingID', 'FullName')
(50, 'Anees Java')
(51, 'Bald Vin')
(52, 'Jay Kon')


# Task 4

In [8]:

procedure = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
SELECT SUM(BillAmount) AS TotalSale,AVG(BillAmount) AS AverageSale,MIN(BillAmount) AS MinimumBillPaid,MAX(BillAmount) AS MaximumBillPaid FROM Orders;
END
"""
## creating stored procedure
cursor.execute(procedure)

## calling stored procedure
cursor.callproc("BasicSalesReport")
results = next(cursor.stored_results())
col = results.column_names
data = results.fetchall()
print(col)
for item in data:
    print(item)

('TotalSale', 'AverageSale', 'MinimumBillPaid', 'MaximumBillPaid')
(Decimal('243'), Decimal('48.6000'), 37, 86)


# Task 5

In [9]:
connection = pool.get_connection()
cursor = connection.cursor(buffered=True)
query6 = """SELECT B.BookingSlot,CONCAT(B.GuestFirstName,' ',B.GuestLastName) AS FullName,E.Name,E.Role FROM bookings B JOIN employees E ON B.EmployeeID = E.EmployeeID
            ORDER BY B.BookingSlot ASC LIMIT 3;"""
cursor.execute(query6)

data = cursor.fetchall()
for item in data:
    print("[{}]".format(item[0]))
    print("[{}]".format(item[1]))
    print("[Assigned to: {} [{}]]".format(item[2],item[3]))
connection.close()


[15:00:00]
[Vanessa McCarthy]
[Assigned to: Giorgos Dioudis [Head Chef]]
[17:30:00]
[Marcos Romero]
[Assigned to: Fatma Kaya [Assistant Chef]]
[18:00:00]
[Anees Java]
[Assigned to: John Millar [Receptionist]]
